In [ ]:
from pyspark.sql import SparkSession


In [ ]:
# Import the necessary library
from pyspark.sql import SparkSession

# Configure the SparkSession to use the local master
spark = (
    SparkSession.builder
        .appName("Instacart-Merge")
        .master("local[*]")
        .config("spark.driver.bindAddress", "127.0.0.1")
        .config("spark.ui.port", "4041")
        .getOrCreate()
)

In [ ]:
# Read source files
aisles   = spark.read.option("header", "true").option("inferSchema", "true").csv("/content/aisles.csv")
depts    = spark.read.option("header", "true").option("inferSchema", "true").csv("/content/departments.csv")
products = spark.read.option("header", "true").option("inferSchema", "true").csv("/content/products.csv")
orders   = spark.read.option("header", "true").option("inferSchema", "true").csv("/content/orders.csv")
train_op = spark.read.option("header", "true").option("inferSchema", "true").csv("/content/order_products__train.csv")
prior_op = spark.read.option("header", "true").option("inferSchema", "true").csv("/content/order_products__prior.csv")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# quick peeks (head / shape analogues)
train_op.show(5, truncate=False)
print(("train_op rows", train_op.count()), ("columns", len(train_op.columns)))

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|1       |49302     |1                |1        |
|1       |11109     |2                |1        |
|1       |10246     |3                |0        |
|1       |49683     |4                |0        |
|1       |43633     |5                |1        |
+--------+----------+-----------------+---------+
only showing top 5 rows

('train_op rows', 1384617) ('columns', 4)


In [ ]:
prior_op.show(5, truncate=False)
print(("prior_op rows", prior_op.count()), ("columns", len(prior_op.columns)))

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|2       |33120     |1                |1        |
|2       |28985     |2                |1        |
|2       |9327      |3                |0        |
|2       |45918     |4                |1        |
|2       |30035     |5                |0        |
+--------+----------+-----------------+---------+
only showing top 5 rows



In [ ]:
#Combine train + prior order‑items
order_products = train_op.unionByName(prior_op)

In [ ]:
order_products.show(5, truncate=False)
print(("order_products rows", order_products.count()), ("columns", len(order_products.columns)))

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|1       |49302     |1                |1        |
|1       |11109     |2                |1        |
|1       |10246     |3                |0        |
|1       |49683     |4                |0        |
|1       |43633     |5                |1        |
+--------+----------+-----------------+---------+
only showing top 5 rows



In [ ]:
# Enrich products with aisle & department names
product_meta = (
    products
        .join(aisles, on="aisle_id", how="inner")
        .join(depts,  on="department_id", how="inner")
)


In [ ]:
# Join order‑items → product metadata → orders
master = (
    order_products
        .join(product_meta, on="product_id", how="inner")
        .join(orders,       on="order_id",   how="inner")
)

In [ ]:
# Get the number of rows and columns in the master DataFrame
num_rows = master.count()
num_cols = len(master.columns)

# Print the shape as a tuple
print((num_rows, num_cols))

(33819103, 15)


In [ ]:
master.show(5, truncate=True)
print(("master rows", master.count()), ("columns", len(master.columns)))

Py4JJavaError: An error occurred while calling o114.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 34.0 failed 1 times, most recent failure: Lost task 0.0 in stage 34.0 (TID 48) (d323adf04bce executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGSc

In [ ]:
type(master.select('product_name'))

pyspark.sql.dataframe.DataFrame

In [ ]:
master.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)
 |-- department_id: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle: string (nullable = true)
 |-- department: string (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- eval_set: string (nullable = true)
 |-- order_number: integer (nullable = true)
 |-- order_dow: integer (nullable = true)
 |-- order_hour_of_day: integer (nullable = true)
 |-- days_since_prior_order: double (nullable = true)



In [ ]:
from pyspark.sql.functions import rand
TARGET_SAMPLE = 2_500_000
RANDOM_SEED   = 42


# Shuffle the DataFrame randomly
shuffled_df = master.orderBy(rand(seed=RANDOM_SEED))

#Get the first 2.5 million rows (no replacement because we just shuffled)
sampled_df = shuffled_df.limit(TARGET_SAMPLE)
sampled_df.cache()

